In [66]:
import tensorflow as tf

from tfx import v1 as tfx
import tensorflow_model_analysis as tfma

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.json_format import MessageToDict
from tfx.proto import example_gen_pb2, trainer_pb2

import os
import pprint
pp = pprint.PrettyPrinter()

import pandas as pd

In [67]:
_cwd = os.getcwd()
# location of the pipeline metadata store
_pipeline_root = os.path.join(_cwd,'pipeline/')

# directory of the raw data files
_data_root = os.path.join(_cwd,'Data/')

# path to the raw training data
_data_filepath = os.path.join(_data_root, 'customer_churn_prediction_dataset.csv')

In [68]:
!head {_data_filepath}

customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0001-XXXX,Male,0,Yes,Yes,5,No,Yes,No,No,Yes,No,No,No,No internet service,Month-to-month,Yes,Credit card,27.43,137.15,Yes
0002-XXXX,Female,0,No,No,42,Yes,No phone service,DSL,Yes,Yes,No,No,No internet service,No internet service,Two year,No,Electronic check,38.28,1607.76,Yes
0003-XXXX,Male,0,No,No,61,Yes,No phone service,No,No,Yes,No,Yes,Yes,No,One year,No,Bank transfer,106.44,6492.84,Yes
0004-XXXX,Male,1,No,Yes,22,Yes,No,No,No internet service,No internet service,Yes,No,No internet service,Yes,Month-to-month,No,Electronic check,92.49,2034.78,No
0005-XXXX,Male,1,Yes,Yes,21,No,No,DSL,No,No,No internet service,No,No,No internet service,One year,No,Bank transfer,19.63,412.23,Yes
0006-XXXX,Female,0,Yes,No,70,Yes,No phone service,No,No 

In [69]:
# Initialize the InteractiveContext with a local sqlite file.
# If you leave `_pipeline_root` blank, then the db will be created in a temporary directory.
# You can safely ignore the warning about the missing config file.
context = InteractiveContext(pipeline_root=_pipeline_root)

In [70]:
# Instantiate ExampleGen with the input CSV dataset
example_gen = tfx.components.CsvExampleGen(input_base=_data_root)

In [71]:
# Execute the component
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 223
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [72]:
# get the artifact object
artifact = example_gen.outputs['examples'].get()[0]

# print split names and uri
print(f'split names: {artifact.split_names}')
print(f'artifact uri: {artifact.uri}')

split names: ["train", "eval"]
artifact uri: /home/kaung_nyo_lwin/AI_Projects/Customer_Churn/pipeline/CsvExampleGen/examples/223


In [73]:
# Get the URI of the output artifact representing the training examples
train_uri = os.path.join(artifact.uri, 'Split-train')

# See the contents of the `train` folder
!ls {train_uri}

data_tfrecord-00000-of-00001.gz


In [74]:
# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

In [75]:
# Define a helper function to get individual examples
def get_records(dataset, num_records):
    '''Extracts records from the given dataset.
    Args:
        dataset (TFRecordDataset): dataset saved by ExampleGen
        num_records (int): number of records to preview
    '''
    
    # initialize an empty list
    records = []
    
    # Use the `take()` method to specify how many records to get
    for tfrecord in dataset.take(num_records):
        
        # Get the numpy property of the tensor
        serialized_example = tfrecord.numpy()
        
        # Initialize a `tf.train.Example()` to read the serialized data
        example = tf.train.Example()
        
        # Read the example data (output is a protocol buffer message)
        example.ParseFromString(serialized_example)
        
        # convert the protocol bufffer message to a Python dictionary
        example_dict = (MessageToDict(example))
        
        # append to the records list
        records.append(example_dict)
        
    return records

In [76]:
# Get 3 records from the dataset
sample_records = get_records(dataset, 3)

# Print the output
pp.pprint(sample_records)

[{'features': {'feature': {'Churn': {'bytesList': {'value': ['WWVz']}},
                           'Contract': {'bytesList': {'value': ['TW9udGgtdG8tbW9udGg=']}},
                           'Dependents': {'bytesList': {'value': ['WWVz']}},
                           'DeviceProtection': {'bytesList': {'value': ['Tm8=']}},
                           'InternetService': {'bytesList': {'value': ['Tm8=']}},
                           'MonthlyCharges': {'floatList': {'value': [27.43]}},
                           'MultipleLines': {'bytesList': {'value': ['WWVz']}},
                           'OnlineBackup': {'bytesList': {'value': ['WWVz']}},
                           'OnlineSecurity': {'bytesList': {'value': ['Tm8=']}},
                           'PaperlessBilling': {'bytesList': {'value': ['WWVz']}},
                           'Partner': {'bytesList': {'value': ['WWVz']}},
                           'PaymentMethod': {'bytesList': {'value': ['Q3JlZGl0IGNhcmQ=']}},
                          

In [77]:
# Instantiate StatisticsGen with the ExampleGen ingested dataset
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])

# Execute the component
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 224
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [78]:
context.show(statistics_gen.outputs['statistics'])

In [79]:
# Instantiate SchemaGen with the StatisticsGen ingested dataset
schema_gen = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    )

# Run the component
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 225
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [80]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Churn',STRING,required,,'Churn'
'Contract',STRING,required,,'Contract'
'Dependents',STRING,required,,'Dependents'
'DeviceProtection',STRING,required,,'DeviceProtection'
'InternetService',STRING,required,,'InternetService'
'MonthlyCharges',FLOAT,required,,-
'MultipleLines',STRING,required,,'MultipleLines'
'OnlineBackup',STRING,required,,'OnlineBackup'
'OnlineSecurity',STRING,required,,'OnlineSecurity'


,Values
Domain,
'Churn',"'No', 'Yes'"
'Contract',"'Month-to-month', 'One year', 'Two year'"
'Dependents',"'No', 'Yes'"
'DeviceProtection',"'No', 'No internet service', 'Yes'"
'InternetService',"'DSL', 'Fiber optic', 'No'"
'MultipleLines',"'No', 'No phone service', 'Yes'"
'OnlineBackup',"'No', 'No internet service', 'Yes'"
'OnlineSecurity',"'No', 'No internet service', 'Yes'"
'PaperlessBilling',"'No', 'Yes'"


In [81]:
# Instantiate ExampleValidator with the StatisticsGen and SchemaGen ingested data
example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])

# Run the component.
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 226
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [82]:
# Visualize the results
context.show(example_validator.outputs['anomalies'])

In [83]:
# Set the constants module filename
_customer_churn_module_file = 'customer_churn_constants.py'

In [84]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Churn',STRING,required,,'Churn'
'Contract',STRING,required,,'Contract'
'Dependents',STRING,required,,'Dependents'
'DeviceProtection',STRING,required,,'DeviceProtection'
'InternetService',STRING,required,,'InternetService'
'MonthlyCharges',FLOAT,required,,-
'MultipleLines',STRING,required,,'MultipleLines'
'OnlineBackup',STRING,required,,'OnlineBackup'
'OnlineSecurity',STRING,required,,'OnlineSecurity'


,Values
Domain,
'Churn',"'No', 'Yes'"
'Contract',"'Month-to-month', 'One year', 'Two year'"
'Dependents',"'No', 'Yes'"
'DeviceProtection',"'No', 'No internet service', 'Yes'"
'InternetService',"'DSL', 'Fiber optic', 'No'"
'MultipleLines',"'No', 'No phone service', 'Yes'"
'OnlineBackup',"'No', 'No internet service', 'Yes'"
'OnlineSecurity',"'No', 'No internet service', 'Yes'"
'PaperlessBilling',"'No', 'Yes'"


In [85]:
%%writefile {_customer_churn_module_file}



VOCAB_FEATURE_DICT = {
    'gender' : 2, 
    'Partner': 2, 
    'Dependents' : 2, 
    'PhoneService' : 2, 
    'MultipleLines': 3, 
    'InternetService' : 3,
    'OnlineSecurity': 3,
    'OnlineBackup' : 3,
    'DeviceProtection' : 3,
    'TechSupport' : 3,
    'StreamingTV' : 3,
    'StreamingMovies' : 3,
    'Contract' : 3,
    'PaperlessBilling' : 2,
    'PaymentMethod' : 4,
    'SeniorCitizen' : 2
}

NUMERIC_FEATURE_KEYS = [
    'MonthlyCharges',
    'TotalCharges'
]

BUCKET_FEATURE_DICT = {'tenure': 4}

# ONE_HOT_DICT = {'SeniorCitizen' : 2}

NUM_OOV_BUCKETS = 1

LABEL_KEY = 'Churn'

# Utility function for renaming the feature
def transformed_name(key):
    return key + '_xf'

Overwriting customer_churn_constants.py


In [86]:
# Set the transform module filename
_customer_churn_transform_module_file = 'customer_churn_transform.py'

In [109]:
%%writefile {_customer_churn_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

import customer_churn_constants

# Unpack the contents of the constants module
_NUMERIC_FEATURE_KEYS = customer_churn_constants.NUMERIC_FEATURE_KEYS
_VOCAB_FEATURE_DICT = customer_churn_constants.VOCAB_FEATURE_DICT
# _ONE_HOT_ONLY = customer_churn_constants.ONE_HOT_DICT
_BUCKET_FEATURE_DICT = customer_churn_constants.BUCKET_FEATURE_DICT
_LABEL_KEY = customer_churn_constants.LABEL_KEY
_NUM_OOV_BUCKETS = customer_churn_constants.NUM_OOV_BUCKETS
_transformed_name = customer_churn_constants.transformed_name
# print(_ONE_HOT_ONLY,_BUCKET_FEATURE_DICT)

# Define the transformations
def preprocessing_fn(inputs):
    """tf.transform's callback function for preprocessing inputs.
    Args:
        inputs: map from feature keys to raw not-yet-transformed features.
    Returns:
        Map from string feature key to transformed feature operations.
    """
    outputs = {}
    
    # # Convert strings to indices and convert to one-hot vectors
    # for key, Depth in _ONE_HOT_ONLY.items():
    #     one_hot = tf.one_hot(inputs[key], Depth)
    #     outputs[_transformed_name(key)] = tf.reshape(one_hot, [-1, Depth])

    # Scale these features to the range [0,1]
    for key in _NUMERIC_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.scale_to_0_1(
            inputs[key])
        
    
     # Convert strings to indices and convert to one-hot vectors
    for key, vocab_size in _VOCAB_FEATURE_DICT.items():
        indices = tft.compute_and_apply_vocabulary(inputs[key], num_oov_buckets=_NUM_OOV_BUCKETS)
        one_hot = tf.one_hot(indices, vocab_size + _NUM_OOV_BUCKETS)
        outputs[_transformed_name(key)] = tf.reshape(one_hot, [-1, vocab_size + _NUM_OOV_BUCKETS])
        
    # Bucketize this feature and convert to one-hot vectors
    for key, num_buckets in _BUCKET_FEATURE_DICT.items():
        indices = tft.bucketize(inputs[key], num_buckets)
        one_hot = tf.one_hot(indices, num_buckets)
        outputs[_transformed_name(key)] = tf.reshape(one_hot, [-1, num_buckets])

    # Convert the label strings to an index
    # outputs[_transformed_name(_LABEL_KEY)] = inputs[_LABEL_KEY]
    outputs[_transformed_name(_LABEL_KEY)] = tft.compute_and_apply_vocabulary(inputs[_LABEL_KEY])
    


    return outputs

Overwriting customer_churn_transform.py


In [110]:
# Ignore TF warning messages
tf.get_logger().setLevel('ERROR')

# Instantiate the Transform component
transform = tfx.components.Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(_customer_churn_transform_module_file))

# Run the component
context.run(transform)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying trainer.py -> build/lib
copying customer_churn_transform.py -> build/lib
copying customer_churn_constants.py -> build/lib
copying tuner.py -> build/lib
copying customer_churn.py -> build/lib
installing to /tmp/tmplwa2no1i
running install
running install_lib
copying build/lib/trainer.py -> /tmp/tmplwa2no1i
copying build/lib/customer_churn_transform.py -> /tmp/tmplwa2no1i
copying build/lib/customer_churn_constants.py -> /tmp/tmplwa2no1i
copying build/lib/tuner.py -> /tmp/tmplwa2no1i
copying build/lib/customer_churn.py -> /tmp/tmplwa2no1i
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
readin

/usr/lib/python3/dist-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(


Processing ./pipeline/_wheels/tfx_user_code_Transform-0.0+8b2b34fabf04f3d9aec3de2c3002577e3d5b0f022736e8e8fde0d118b189c723-py3-none-any.whl


Processing ./pipeline/_wheels/tfx_user_code_Transform-0.0+8b2b34fabf04f3d9aec3de2c3002577e3d5b0f022736e8e8fde0d118b189c723-py3-none-any.whl


Processing ./pipeline/_wheels/tfx_user_code_Transform-0.0+8b2b34fabf04f3d9aec3de2c3002577e3d5b0f022736e8e8fde0d118b189c723-py3-none-any.whl


ExecutionResult(
    component_id: Transform
    execution_id: 234
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [111]:
# Get the uri of the transform graph
transform_graph_uri = transform.outputs['transform_graph'].get()[0].uri

# List the subdirectories under the uri
os.listdir(transform_graph_uri)

['transformed_metadata', 'metadata', 'transform_fn']

In [112]:
# Get the URI of the output artifact representing the transformed examples
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
transformed_dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

In [113]:
# Get 3 records from the dataset
sample_records_xf = get_records(transformed_dataset, 1)

# Print the output
pp.pprint(len(sample_records_xf[0]['features']['feature']))
pp.pprint(sample_records_xf)

tf.data.experimental.cardinality(transformed_dataset)

20
[{'features': {'feature': {'Churn_xf': {'int64List': {'value': ['1']}},
                           'Contract_xf': {'floatList': {'value': [1.0,
                                                                   0.0,
                                                                   0.0,
                                                                   0.0]}},
                           'Dependents_xf': {'floatList': {'value': [1.0,
                                                                     0.0,
                                                                     0.0]}},
                           'DeviceProtection_xf': {'floatList': {'value': [0.0,
                                                                           1.0,
                                                                           0.0,
                                                                           0.0]}},
                           'InternetService_xf': {'floatList': {'value': [1.0,
       

<tf.Tensor: shape=(), dtype=int64, numpy=-2>

In [114]:
# Declare name of module file
_tuner_module_file = 'tuner.py'

In [115]:
%%writefile {_tuner_module_file}

# Define imports
from kerastuner.engine import base_tuner
import kerastuner as kt
from tensorflow import keras
from typing import NamedTuple, Dict, Text, Any, List
from tfx.components.trainer.fn_args_utils import FnArgs, DataAccessor
import tensorflow as tf
import tensorflow_transform as tft

import customer_churn_constants

# Unpack the contents of the constants module
_NUMERIC_FEATURE_KEYS = customer_churn_constants.NUMERIC_FEATURE_KEYS
_VOCAB_FEATURE_DICT = customer_churn_constants.VOCAB_FEATURE_DICT
# _ONE_HOT_ONLY = customer_churn_constants.ONE_HOT_DICT
_BUCKET_FEATURE_DICT = customer_churn_constants.BUCKET_FEATURE_DICT
_NUM_OOV_BUCKETS = customer_churn_constants.NUM_OOV_BUCKETS
_transformed_name = customer_churn_constants.transformed_name
_LABEL_KEY = _transformed_name(customer_churn_constants.LABEL_KEY)
# print(_ONE_HOT_ONLY,_BUCKET_FEATURE_DICT)

# Declare namedtuple field names
TunerFnResult = NamedTuple('TunerFnResult', [('tuner', base_tuner.BaseTuner),
                                             ('fit_kwargs', Dict[Text, Any])])

# # Input key
# _FEATURE_KEY = ['Contract_xf', 'Dependents_xf', 'DeviceProtection_xf', 'InternetService_xf', 
#                 'MonthlyCharges_xf', 'MultipleLines_xf', 'OnlineBackup_xf', 'OnlineSecurity_xf', 
#                 'PaperlessBilling_xf', 'Partner_xf', 'PaymentMethod_xf', 'PhoneService_xf', 'SeniorCitizen_xf', 
#                 'StreamingMovies_xf', 'StreamingTV_xf', 'TechSupport_xf', 'TotalCharges_xf', 'gender_xf', 'tenure_xf']

# # Label key
# _LABEL_KEY = 'Churn_xf'

# Callback for the search strategy
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=5,baseline = 0.75,min_delta = 0.001)


def _gzip_reader_fn(filenames):
  '''Load compressed dataset
  
  Args:
    filenames - filenames of TFRecords to load

  Returns:
    TFRecordDataset loaded from the filenames
  '''

  # Load the dataset. Specify the compression type since it is saved as `.gz`
  return tf.data.TFRecordDataset(filenames, compression_type='GZIP')
  

def _input_fn(file_pattern,
              tf_transform_output,
              num_epochs=None,
              batch_size=4) -> tf.data.Dataset:
  '''Create batches of features and labels from TF Records

  Args:
    file_pattern - List of files or patterns of file paths containing Example records.
    tf_transform_output - transform output graph
    num_epochs - Integer specifying the number of times to read through the dataset. 
            If None, cycles through the dataset forever.
    batch_size - An int representing the number of records to combine in a single batch.

  Returns:
    A dataset of dict elements, (or a tuple of dict elements and label). 
    Each dict maps feature keys to Tensor or SparseTensor objects.
  '''

  # Get feature specification based on transform output
  transformed_feature_spec = (
      tf_transform_output.transformed_feature_spec().copy())
  
  # Create batches of features and labels
  dataset = tf.data.experimental.make_batched_features_dataset(
      file_pattern=file_pattern,
      batch_size=batch_size,
      features=transformed_feature_spec,
      reader=_gzip_reader_fn,
      num_epochs=num_epochs,
      label_key=_LABEL_KEY)
  
  return dataset


def model_builder(hp):
  '''
  Builds the model and sets up the hyperparameters to tune.

  Args:
    hp - Keras tuner object

  Returns:
    model with hyperparameters to tune
  '''

  # Define input layers for numeric keys
  input_numeric = [
      tf.keras.layers.Input(name=_transformed_name(colname), shape=(1,), dtype=tf.float32)
      for colname in _NUMERIC_FEATURE_KEYS
  ]

  # Define input layers for vocab keys
  input_categorical = [
      tf.keras.layers.Input(name=_transformed_name(colname), shape=(vocab_size + _NUM_OOV_BUCKETS,), dtype=tf.float32)
      for colname, vocab_size in _VOCAB_FEATURE_DICT.items()
  ]

  # Define input layers for bucket key
  input_categorical += [
      tf.keras.layers.Input(name=_transformed_name(colname), shape=(num_buckets,), dtype=tf.float32)
      for colname, num_buckets in _BUCKET_FEATURE_DICT.items()
  ]
  
  inputs = input_numeric + input_categorical
  
  d = keras.layers.concatenate(inputs)
  # model.add(keras.layers.Flatten())

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=1024, step=32)
  d = keras.layers.Dense(units=hp_units, activation='relu')(d)

  # Add next layers
  d = keras.layers.Dropout(0.2)(d)
  outputs = keras.layers.Dense(1, activation='sigmoid')(d)

  model = keras.Model(inputs=inputs, outputs=outputs)

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss='binary_crossentropy',
                metrics='binary_accuracy')
  model.summary()
  return model

def tuner_fn(fn_args: FnArgs) -> TunerFnResult:
  """Build the tuner using the KerasTuner API.
  Args:
    fn_args: Holds args as name/value pairs.

      - working_dir: working dir for tuning.
      - train_files: List of file paths containing training tf.Example data.
      - eval_files: List of file paths containing eval tf.Example data.
      - train_steps: number of train steps.
      - eval_steps: number of eval steps.
      - schema_path: optional schema of the input data.
      - transform_graph_path: optional transform graph produced by TFT.
  
  Returns:
    A namedtuple contains the following:
      - tuner: A BaseTuner that will be used for tuning.
      - fit_kwargs: Args to pass to tuner's run_trial function for fitting the
                    model , e.g., the training and validation dataset. Required
                    args depend on the above tuner's implementation.
  """

  # Define tuner search strategy
  tuner = kt.Hyperband(model_builder,
                     objective='val_binary_accuracy',
                     max_epochs=30,
                     factor=3,
                     directory=fn_args.working_dir,
                    #  max_retries_per_trial = 5,
                    #  max_consecutive_failed_trials = 10,
                     project_name='kt_hyperband')
  print(tuner.oracle.max_trials)
  # Load transform output
  tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

  # Use _input_fn() to extract input features and labels from the train and val set
  train_set = _input_fn(fn_args.train_files[0], tf_transform_output)
  val_set = _input_fn(fn_args.eval_files[0], tf_transform_output)


  return TunerFnResult(
      tuner=tuner,
      fit_kwargs={ 
          "callbacks":[stop_early],
          'x': train_set,
          'validation_data': val_set,
          'steps_per_epoch': fn_args.train_steps,
          'validation_steps': fn_args.eval_steps
      }
  )

Overwriting tuner.py


In [116]:
# Setup the Tuner component
tuner = tfx.components.Tuner(
    module_file=_tuner_module_file,
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=60),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=15)
    )

In [117]:
# Run the component. This will take around 10 minutes to run.
# When done, it will summarize the results and show the 10 best trials.
context.run(tuner, enable_cache=False)


Trial 90 Complete [00h 00m 05s]
val_binary_accuracy: 0.5333333611488342

Best val_binary_accuracy So Far: 0.6666666865348816
Total elapsed time: 00h 07m 53s
Results summary
Results in /home/kaung_nyo_lwin/AI_Projects/Customer_Churn/pipeline/.temp/235/kt_hyperband
Showing 10 best trials
Trial summary
Hyperparameters:
units: 928
learning_rate: 0.0001
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 3
tuner/round: 0
Score: 0.6666666865348816
Trial summary
Hyperparameters:
units: 480
learning_rate: 0.01
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.6499999761581421
Trial summary
Hyperparameters:
units: 128
learning_rate: 0.0001
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 3
tuner/round: 0
Score: 0.6333333253860474
Trial summary
Hyperparameters:
units: 928
learning_rate: 0.0001
tuner/epochs: 4
tuner/initial_epoch: 2
tuner/bracket: 3
tuner/round: 1
tuner/trial_id: 0001
Score: 0.6333333253860474
Trial summary
Hyperparameters:
units: 928
learnin

ExecutionResult(
    component_id: Tuner
    execution_id: 235
    outputs:
        best_hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        tuner_results: OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [118]:
# Declare trainer module file
_trainer_module_file = 'trainer.py'

In [133]:
%%writefile {_trainer_module_file}

from typing import List, Text

import tensorflow as tf
import tensorflow_transform as tft
from tensorflow_transform.tf_metadata import schema_utils

from tfx.components.trainer.fn_args_utils import DataAccessor, FnArgs
from tfx_bsl.public.tfxio import TensorFlowDatasetOptions
import keras

# import same constants from transform module
import customer_churn_constants
# Unpack the contents of the constants module
_NUMERIC_FEATURE_KEYS = customer_churn_constants.NUMERIC_FEATURE_KEYS
_VOCAB_FEATURE_DICT = customer_churn_constants.VOCAB_FEATURE_DICT
# _ONE_HOT_ONLY = customer_churn_constants.ONE_HOT_DICT
_BUCKET_FEATURE_DICT = customer_churn_constants.BUCKET_FEATURE_DICT
_NUM_OOV_BUCKETS = customer_churn_constants.NUM_OOV_BUCKETS
_transformed_name = customer_churn_constants.transformed_name
_LABEL_KEY = _transformed_name(customer_churn_constants.LABEL_KEY)

def _gzip_reader_fn(filenames):
  '''Load compressed dataset
  
  Args:
    filenames - filenames of TFRecords to load

  Returns:
    TFRecordDataset loaded from the filenames
  '''

  # Load the dataset. Specify the compression type since it is saved as `.gz`
  return tf.data.TFRecordDataset(filenames, compression_type='GZIP')
  

def _input_fn(file_pattern,
              tf_transform_output,
              num_epochs=None,
              batch_size=32) -> tf.data.Dataset:
  '''Create batches of features and labels from TF Records

  Args:
    file_pattern - List of files or patterns of file paths containing Example records.
    tf_transform_output - transform output graph
    num_epochs - Integer specifying the number of times to read through the dataset. 
            If None, cycles through the dataset forever.
    batch_size - An int representing the number of records to combine in a single batch.

  Returns:
    A dataset of dict elements, (or a tuple of dict elements and label). 
    Each dict maps feature keys to Tensor or SparseTensor objects.
  '''
  transformed_feature_spec = (
      tf_transform_output.transformed_feature_spec().copy())
  
  dataset = tf.data.experimental.make_batched_features_dataset(
      file_pattern=file_pattern,
      batch_size=batch_size,
      features=transformed_feature_spec,
      reader=_gzip_reader_fn,
      num_epochs=num_epochs,
      label_key=_LABEL_KEY)
  
  return dataset


def model_builder(hp):
  '''
  Builds the model and sets up the hyperparameters to tune.

  Args:
    hp - Keras tuner object

  Returns:
    model with hyperparameters to tune
  '''

  # Define input layers for numeric keys
  input_numeric = [
      tf.keras.layers.Input(name=_transformed_name(colname), shape=(1,), dtype=tf.float32)
      for colname in _NUMERIC_FEATURE_KEYS
  ]

  # Define input layers for vocab keys
  input_categorical = [
      tf.keras.layers.Input(name=_transformed_name(colname), shape=(vocab_size + _NUM_OOV_BUCKETS,), dtype=tf.float32)
      for colname, vocab_size in _VOCAB_FEATURE_DICT.items()
  ]

  # Define input layers for bucket key
  input_categorical += [
      tf.keras.layers.Input(name=_transformed_name(colname), shape=(num_buckets,), dtype=tf.float32)
      for colname, num_buckets in _BUCKET_FEATURE_DICT.items()
  ]
  
  inputs = input_numeric + input_categorical
  
  d = keras.layers.concatenate(inputs)
  # model.add(keras.layers.Flatten())

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.get('units')
  d = keras.layers.Dense(units=hp_units, activation='relu')(d)

  # Add next layers
  d = keras.layers.Dropout(0.2)(d)
  outputs = keras.layers.Dense(1, activation='sigmoid')(d)

  model = keras.Model(inputs=inputs, outputs=outputs)

  # Get the learning rate from the Tuner results
  hp_learning_rate = hp.get('learning_rate')

  # Setup model for training
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss='binary_crossentropy',
                metrics='binary_accuracy')

  # Print the model summary
  model.summary()
  
  return model

def _get_tf_examples_serving_signature(model, tf_transform_output):
  """Returns a serving signature that accepts `tensorflow.Example`."""

  # Get the transformation graph
  model.tft_layer_inference = tf_transform_output.transform_features_layer()

  @tf.function(input_signature=[
      tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
  ])
  def serve_tf_examples_fn(serialized_tf_example):
    """Returns the output to be used in the serving signature."""
    
    # Get pre-transform feature spec
    raw_feature_spec = tf_transform_output.raw_feature_spec()
    
    # Remove label feature since these will not be present at serving time.
    #raw_feature_spec.pop('Churn')
    
    # Parse raw examples into a dictionary of tensors matching the feature spec
    raw_features = tf.io.parse_example(serialized_tf_example, raw_feature_spec)
    
    # Transform the raw examples using the transform graph
    transformed_features = model.tft_layer_inference(raw_features)

    # Get predictions using the transformed features
    outputs = model(transformed_features)

    return {'outputs': outputs}

  return serve_tf_examples_fn

def eval_input_receiver_fn(tf_transform_output):
  """Returns a serving signature that accepts `tensorflow.Example`."""

  # Get the transformation graph
  raw_feature_spec = tf_transform_output.raw_feature_spec()
  
  serialized_tf_example = tf.compat.v1.placeholder(
      dtype=tf.string, shape=[None], name='input_example_tensor')

  # Add a parse_example operator to the tensorflow graph, which will parse
  # raw, untransformed, tf examples.
  features = tf.io.parse_example(
      serialized=serialized_tf_example, features=raw_feature_spec)

  # Now that we have our raw examples, process them through the tf-transform
  # function computed during the preprocessing step.
  transformed_features = tf_transform_output.transform_raw_features(
      features)

  # The key name MUST be 'examples'.
  receiver_tensors = {'examples': serialized_tf_example}

  # NOTE: Model is driven by transformed features (since training works on the
  # materialized output of TFT, but slicing will happen on raw features.
  features.update(transformed_features)

  return tfma.export.EvalInputReceiver(
      features=features,
      receiver_tensors=receiver_tensors,
      labels=transformed_features[_LABEL_KEY])
  
def export_serving_model(tf_transform_output, model, output_dir):
  """Exports a keras model for serving.
  Args:
    tf_transform_output: Wrapper around output of tf.Transform.
    model: A keras model to export for serving.
    output_dir: A directory where the model will be exported to.
  """
  # The layer has to be saved to the model for keras tracking purpases.
  model.tft_layer = tf_transform_output.transform_features_layer()

  signatures = {
      'serving_default':
          _get_tf_examples_serving_signature(model, tf_transform_output)
  }

  model.save(output_dir, save_format='tf', signatures=signatures)

def run_fn(fn_args: FnArgs) -> None:
  """Defines and trains the model.
  Args:
    fn_args: Holds args as name/value pairs. Refer here for the complete attributes: 
    https://www.tensorflow.org/tfx/api_docs/python/tfx/components/trainer/fn_args_utils/FnArgs#attributes
  """
  
  # Load transform output
  tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
  
  # Create batches of data good for 10 epochs
  train_set = _input_fn(fn_args.train_files[0], tf_transform_output, 10)
  val_set = _input_fn(fn_args.eval_files[0], tf_transform_output, 10)

  # Load best hyperparameters
  hp = fn_args.hyperparameters.get('values')
  print(hp)
  # Build the model
  model = model_builder(hp)
  # Callback for TensorBoard
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=fn_args.model_run_dir, update_freq='batch')
  # Train the model
  model.fit(
      x=train_set,
      validation_data=val_set,
      callbacks=[tensorboard_callback]
      )
  
  export_serving_model(tf_transform_output, model, fn_args.serving_model_dir)
  


Overwriting trainer.py


In [134]:
# Setup the Trainer component
trainer = tfx.components.Trainer(
    module_file=_trainer_module_file,
    examples=transform.outputs['transformed_examples'],
    hyperparameters=tuner.outputs['best_hyperparameters'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train']),
    eval_args=trainer_pb2.EvalArgs(splits=['eval']))

In [135]:
context.run(trainer, enable_cache=False)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying trainer.py -> build/lib
copying customer_churn_transform.py -> build/lib
copying customer_churn_constants.py -> build/lib
copying tuner.py -> build/lib
copying customer_churn.py -> build/lib
installing to /tmp/tmplzstst2d
running install
running install_lib
copying build/lib/trainer.py -> /tmp/tmplzstst2d
copying build/lib/customer_churn_transform.py -> /tmp/tmplzstst2d
copying build/lib/customer_churn_constants.py -> /tmp/tmplzstst2d
copying build/lib/tuner.py -> /tmp/tmplzstst2d
copying build/lib/customer_churn.py -> /tmp/tmplzstst2d
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest

/usr/lib/python3/dist-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(


Processing ./pipeline/_wheels/tfx_user_code_Trainer-0.0+2644387d30a2bc8a72ec9208daa5e79d41a7fb2a4991db23393dd7a41086f778-py3-none-any.whl


{'units': 928, 'learning_rate': 0.0001, 'tuner/epochs': 2, 'tuner/initial_epoch': 0, 'tuner/bracket': 3, 'tuner/round': 0}
Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 MonthlyCharges_xf (InputLa  [(None, 1)]                  0         []                            
 yer)                                                                                             
                                                                                                  
 TotalCharges_xf (InputLaye  [(None, 1)]                  0         []                            
 r)                                                                                               
                                                                                                  
 gender_xf (InputLayer)      [(None, 3)]                  0         

/usr/local/lib/python3.10/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['Churn_xf'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


ExecutionResult(
    component_id: Trainer
    execution_id: 242
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [137]:
eval_config = tfma.EvalConfig(
    model_specs=[
        # This assumes a serving model with signature 'serving_default'.
        tfma.ModelSpec(
            signature_name='serving_default',
            label_key='Churn_xf'
            )
        ],
    metrics_specs=[
        tfma.MetricsSpec(
            # The metrics added here are in addition to those saved with the
            # model (assuming either a keras model or EvalSavedModel is used).
            # Any metrics added into the saved model (for example using
            # model.compile(..., metrics=[...]), etc) will be computed
            # automatically.
            # To add validation thresholds for metrics saved with the model,
            # add them keyed by metric name to the thresholds map.
            metrics=[
                tfma.MetricConfig(class_name='ExampleCount'),
                tfma.MetricConfig(class_name='BinaryAccuracy',
                      threshold=tfma.MetricThreshold(
                          value_threshold=tfma.GenericValueThreshold(
                              lower_bound={'value': 0.5}),
                              # Change threshold will be ignored if there is no
                              # baseline model resolved from MLMD (first run).
                              change_threshold=tfma.GenericChangeThreshold(
                                  direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                                  absolute={'value': -1e-10}
                                  )
                              )
                      )
            ]
        )
    ],
    slicing_specs=[
        # An empty slice spec means the overall slice, i.e. the whole dataset.
        tfma.SlicingSpec(),
    ])

In [130]:
# Setup the Resolver node to find the latest blessed model
model_resolver = tfx.dsl.Resolver(
      strategy_class=tfx.dsl.experimental.LatestBlessedModelStrategy,
      model=tfx.dsl.Channel(type=tfx.types.standard_artifacts.Model),
      model_blessing=tfx.dsl.Channel(
          type=tfx.types.standard_artifacts.ModelBlessing)).with_id(
              'latest_blessed_model_resolver')

# Run the Resolver node
context.run(model_resolver)

ExecutionResult(
    component_id: latest_blessed_model_resolver
    execution_id: 240
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [131]:
# Load Resolver outputs
model_resolver.outputs['model']

.type_name,Model
._artifacts,[]


In [139]:
# Setup and run the Evaluator component
evaluator = tfx.components.Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
context.run(evaluator, enable_cache=False)